In [2]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import os
import pandas as pd
import numpy as np
from predict import preprocess, train_test_split, df_yielder
from sklearn.preprocessing import MinMaxScaler

TRAIN_RATIO = 0.2
TIMESTEP = 12


In [3]:
def create_model(input_shape):
    
    model = Sequential()
    model.add(LSTM(units=50, input_shape=input_shape))
    model.add(Dense(units=1))
    model.compile(optimizer='adam',loss='mean_squared_error')
    
    return model

In [13]:
df = preprocess(pd.read_csv('data/yf_attributes/AAPL.csv'))

scaler_X = MinMaxScaler(feature_range=(0, 1))
scaler_y = MinMaxScaler(feature_range=(0, 1))

scaled_X = scaler_X.fit_transform(df)
scaled_y = scaler_y.fit_transform(df[['Close']])


In [5]:
def create_sequences(data, timestep):
    X, y = [], []
    for i in range(len(data) - timestep):
        X.append(data[i:i + timestep]) #Using all data between timesteps
        y.append(data[i + timestep, 4]) #Using next timesteps close data as target
    
    return np.array(X), np.array(y)

In [6]:

X, y = create_sequences(scaled_X, TIMESTEP)

split_point = int(TRAIN_RATIO * len(X))
X_train, X_test = X[:split_point], X[split_point:]
y_train, y_test = y[:split_point], y[split_point:]

#Shape of X will be (data_length, timesteps, num_features)


In [7]:
model = create_model((X.shape[1], X.shape[2]))
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)



Epoch 1/50

3/3 [==============================] - 2s 11ms/step - loss: 2.1101e-04
Epoch 2/50
3/3 [==============================] - 0s 10ms/step - loss: 1.9020e-04
Epoch 3/50
3/3 [==============================] - 0s 10ms/step - loss: 4.2352e-05
Epoch 4/50
3/3 [==============================] - 0s 11ms/step - loss: 8.6329e-05
Epoch 5/50
3/3 [==============================] - 0s 10ms/step - loss: 1.1754e-05
Epoch 6/50
3/3 [==============================] - 0s 10ms/step - loss: 3.6451e-05
Epoch 7/50
3/3 [==============================] - 0s 10ms/step - loss: 1.3048e-05
Epoch 8/50
3/3 [==============================] - 0s 9ms/step - loss: 1.1983e-05
Epoch 9/50
3/3 [==============================] - 0s 9ms/step - loss: 1.5341e-05
Epoch 10/50
3/3 [==============================] - 0s 9ms/step - loss: 3.5136e-06
Epoch 11/50
3/3 [==============================] - 0s 10ms/step - loss: 1.0155e-05
Epoch 12/50
3/3 [==============================] - 0s 9ms/step - loss: 3.1787e-06
Epoch 13/50
3/

In [8]:
loss = model.evaluate(X_test, y_test)
print("Test Loss:", loss)

12/12 [==============================] - 1s 4ms/step - loss: 0.0613
Test Loss: 0.06129630282521248


In [15]:
last_sequence = X_test[-1].reshape(1, X_test[-1].shape[0], X_test[-1].shape[1])

next_close = model.predict(last_sequence)

descaled_next_close = scaler_y.inverse_transform(next_close)

print(descaled_next_close)
#TODO WHY PREDICTION IS SO OFF?


1/1 [==============================] - 0s 24ms/step
[[30.79161]]
